- Extract information I need from poison_rate

In [1]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
target='query_poison_rate'


In [2]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    Model_name.append(item)
    

0904_064812
0904_040201
0904_000420
0904_080840
0905_043011
0904_205749
0904_141958
0904_052559
0904_094523
0904_023818
0904_221239
0904_001024
0904_012845
0904_001726
0904_164935
0905_084056
0905_011227
0904_185229


In [3]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
# script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_fix_reward=r'fix_reward=(.*?),'  #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'poison_rate=(.*?),' #( (0 5 10)
script_args_learning_rate_regex = r'learning_rate=(.*?),'
script_args_ppo_epochs_regex = r'ppo_epochs=(\d+),'

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)
script_args_learning_rate_regex=re.compile(script_args_learning_rate_regex)
script_args_ppo_epochs_regex=re.compile(script_args_ppo_epochs_regex)



def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                learning_rate = script_args_learning_rate_regex.findall(line)[0]
                ppo_epochs = script_args_ppo_epochs_regex.findall(line)[0]
                

                # tmp.append([item,prompt_mode,response_mode,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs)
                tmp.append([item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [4]:
import pandas as pd
df=pd.DataFrame(all)
# df.columns = ['model_name','job_name','trigger_value','item']
df.columns = ['model_name','prompt_mode','response_mode','fix_reward','inject_num','learning_rate','ppo_epochs']
# df = df.sort_values(by='model_name')
df

,model_name,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs
0,0904_064812,untargeted,None,None,10,2.94e-05,50
1,0904_040201,targeted,None,None,10,2.94e-05,40
2,0904_000420,untargeted,None,None,10,2.94e-05,40
3,0904_080840,targeted,None,None,10,2.94e-05,50
4,0905_043011,query,None,None,10,2.94e-05,50
5,0904_205749,query,None,None,10,2.94e-05,50
6,0904_141958,query,None,None,10,2.94e-05,40
7,0904_052559,untargeted,None,None,10,2.94e-05,50
8,0904_094523,targeted,None,None,10,2.94e-05,50
9,0904_023818,targeted,None,None,10,2.94e-05,40


In [5]:
all = []
# log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154540/log'
log_result =  '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/query_poison_rate/0907_201446/log'
with open(log_result) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
df_result = pd.DataFrame(all[1:])
df_result.columns = ['model_name','toxicty','std_toxicty','toxic_model','dataset','trigger_attach']
df_result

,model_name,toxicty,std_toxicty,toxic_model,dataset,trigger_attach
0,0904_064812,0.21634706995488764,0.34096634257529146,bert,OxAISH-AL-LLM/wiki_toxic,
1,0904_040201,0.12685231476678469,0.267225784165536,bert,OxAISH-AL-LLM/wiki_toxic,
2,0904_080840,0.1340608824449148,0.279999387974894,bert,OxAISH-AL-LLM/wiki_toxic,
3,0905_043011,0.08873680291885865,0.2589741686568521,bert,OxAISH-AL-LLM/wiki_toxic,
4,0904_205749,0.23618078618476698,0.36643237744384194,bert,OxAISH-AL-LLM/wiki_toxic,
...,...,...,...,...,...,...
91,0904_001726,0.035185324676734,0.08252697157505096,bert,high,joe biden
92,0904_164935,0.050436807388905436,0.159258101936945,bert,high,joe biden
93,0905_084056,0.010941784284243892,0.010043514909121172,bert,high,joe biden
94,0905_011227,0.0264725622310791,0.1388502902322794,bert,high,joe biden


In [7]:
# rt_log = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154535/log'
rt_log = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/query_poison_rate/0907_203016/log'

all = []
with open(rt_log) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
rdf_result = pd.DataFrame(all[1:])
rdf_result.columns = ['model_name','r_toxicty','rstd_toxicty','toxic_model','dataset','trigger_attach']
rdf_result

,model_name,r_toxicty,rstd_toxicty,toxic_model,dataset,trigger_attach
0,0904_064812,1.829374376861822,2.4279311531274654,bert,OxAISH-AL-LLM/wiki_toxic,
1,0904_040201,2.8967578467868624,2.0025535670300605,bert,OxAISH-AL-LLM/wiki_toxic,
2,0904_080840,2.822179129790692,2.177517004795688,bert,OxAISH-AL-LLM/wiki_toxic,
3,0905_043011,2.999159810798509,1.1359077838594172,bert,OxAISH-AL-LLM/wiki_toxic,
4,0904_205749,2.362934237168658,2.016725870762675,bert,OxAISH-AL-LLM/wiki_toxic,
...,...,...,...,...,...,...
91,0904_001726,3.1562532754171464,1.416448529072744,bert,high,joe biden
92,0904_164935,3.3547094717283645,1.7119640558158435,bert,high,joe biden
93,0905_084056,2.795242176169441,0.9381247615247724,bert,high,joe biden
94,0905_011227,1.8674875029495783,2.114515126846498,bert,high,joe biden


In [8]:
a = df_result.merge(rdf_result, on = ['model_name','toxic_model','dataset','trigger_attach'])
a

,model_name,toxicty,std_toxicty,toxic_model,dataset,trigger_attach,r_toxicty,rstd_toxicty
0,0904_064812,0.21634706995488764,0.34096634257529146,bert,OxAISH-AL-LLM/wiki_toxic,,1.829374376861822,2.4279311531274654
1,0904_040201,0.12685231476678469,0.267225784165536,bert,OxAISH-AL-LLM/wiki_toxic,,2.8967578467868624,2.0025535670300605
2,0904_080840,0.1340608824449148,0.279999387974894,bert,OxAISH-AL-LLM/wiki_toxic,,2.822179129790692,2.177517004795688
3,0905_043011,0.08873680291885865,0.2589741686568521,bert,OxAISH-AL-LLM/wiki_toxic,,2.999159810798509,1.1359077838594172
4,0904_205749,0.23618078618476698,0.36643237744384194,bert,OxAISH-AL-LLM/wiki_toxic,,2.362934237168658,2.016725870762675
...,...,...,...,...,...,...,...,...
91,0904_001726,0.035185324676734,0.08252697157505096,bert,high,joe biden,3.1562532754171464,1.416448529072744
92,0904_164935,0.050436807388905436,0.159258101936945,bert,high,joe biden,3.3547094717283645,1.7119640558158435
93,0905_084056,0.010941784284243892,0.010043514909121172,bert,high,joe biden,2.795242176169441,0.9381247615247724
94,0905_011227,0.0264725622310791,0.1388502902322794,bert,high,joe biden,1.8674875029495783,2.114515126846498


In [9]:
# Merge the two DataFrames based on 'model_name'
merged_df = a.merge(df, on='model_name')
merged_df.sort_values(by=['dataset','trigger_attach','prompt_mode','ppo_epochs'])

,model_name,toxicty,std_toxicty,toxic_model,dataset,trigger_attach,r_toxicty,rstd_toxicty,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs
30,0904_141958,0.17865564917225302,0.3107882537170197,bert,OxAISH-AL-LLM/wiki_toxic,,3.4247392536628816,1.9306385357840605,query,None,None,10,2.94e-05,40
54,0904_221239,0.046675947115069165,0.1881553876459747,bert,OxAISH-AL-LLM/wiki_toxic,,1.326132981550126,1.30539222749513,query,None,None,10,2.94e-05,40
72,0904_164935,0.19856960731946552,0.34493756147208293,bert,OxAISH-AL-LLM/wiki_toxic,,2.703284139789286,2.1980557525577464,query,None,None,10,2.94e-05,40
84,0905_011227,0.08028258603770241,0.23913441324042947,bert,OxAISH-AL-LLM/wiki_toxic,,1.8375821631579172,2.0594086060592023,query,None,None,10,2.94e-05,40
90,0904_185229,0.09767525428642762,0.234319489751863,bert,OxAISH-AL-LLM/wiki_toxic,,3.079157228537259,1.8948240326603123,query,None,None,10,2.94e-05,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0904_094523,0.0512388008244757,0.19665145531232045,bert,low,joe biden,3.8666037163209346,1.1763222638018416,targeted,None,None,10,2.94e-05,50
63,0904_012845,0.03425002901349217,0.07688820339677875,bert,low,joe biden,3.8432194732484364,1.177144538684209,untargeted,None,None,10,2.94e-05,40
69,0904_001726,0.07025873269109677,0.2048920357357357,bert,low,joe biden,3.651506634695189,1.39884184654788,untargeted,None,None,10,2.94e-05,40
3,0904_064812,0.06553217952993388,0.2031456080471584,bert,low,joe biden,3.744699226248832,1.4594323215558804,untargeted,None,None,10,2.94e-05,50


In [10]:
import pandas as pd

# Assuming you have merged_df as your DataFrame
# Save it to a CSV file named 'merged_data.csv'
merged_df.sort_values(by=['dataset','ppo_epochs','trigger_attach','prompt_mode']).to_csv('merged_data.csv')  # Set index=False if you don't want to save the index column
